<a href="https://colab.research.google.com/github/ishitadebnath/ML-gpe/blob/master/Phenotype_Days_to_Flowering_Detailed_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Phenotype Prediction: Days to Flowering**

The phenotype of an organism is the result of an interplay betweeen its genetic composition and the environment. Due to the gradual change of the climate, it becomes essential to understand the influence of genotype and the environment on plant phenotypes. Here, we intend to develop a machine learning (ML) model, which could predict the phenotype: Days to Flowering. Currently, our model can predict the phenotypic attributes in Sorghum bicolor and we intend to solve this problem in other plant species as well. The model takes the genetic data of the plants and environmental data as input. The gene data is based on Single Nucleotide Polymorphisms (SNP). It is filtered by entropy and added as features in the model. The environmental data consists of location of the plant (range and column of the subplot) and weather parameters from the weather station. Also, sowing date of the plants and window size are required to give as input parameter to the model. 

Here, we compare verious machine learning models on these dataset with 5-fold cross-validation in order select the final model. The models were evaluated based on Root Mean Squared Error (RMSE) and we find Extreme Gradient Boosting (XgBoost) regressor to perform best on these dataset. Therefore, finally the script outputs a trained model with XgBoost regressor, which can be used to predict the phenotype.    

**Dataset:**
The following dataset is required in a particular format.

**1. Trait Data of Plant Species:** Each observation in this dataset denotes the location (range and column) of the subplot, cultivar information, the date and days of flowering.

*Columns: plot, range, column, scientificname, genotype, treatment, blocking_height, method, date_of_flowering, days_to_flowering, gdd_to_flowering, method_type*

**2. Gene Data Filtered by Entropy:** The gene data comprises of SNPS from 4.4k genes present in 362 cultivars. The gene data is clustered using k-means with Euclidean distances for 30 gene clusters. Thus, the number of SNPs that each cultivar has in 30 gene clusters is added as features in the model.

**3. Weather Data:** The weather data is obtained from a local weather station and the parameters include temperature, relative humidity and vapor pressure deficit.

*Columns: date, day_of_year, temp_min, temp_max, temp_mean, gdd, rh_min, rh_max, rh_mean, vpd_mean, precip, precip_cumulative, first_water_deficit_treatment, second_water_deficit_treatment*

**4. Sowing Date:** The date at which the seeds of the plants were sowed.

**5. Window Size:** The number of days from the sowing date for which the environmental variables are required to be used to train the model.

The following script contains all the steps, that we followed to build the model for Sorghum Bicolor.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
from scipy import stats
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, GroupKFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LassoLars
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import statistics
import os
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [ ]:
# This code block is not required to be executed if any github URL or a path on local machine is being provides as the path for the data. 
# Since, currently we are using Google Colab notebook and data is store on Google Drive, hence the drive location is mounted over here. 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mac=pd.read_csv("/content/drive/MyDrive/GPE/MAC Season 4/mac_season_4_days_gdd_to_flowering.csv" , delimiter=",")
mac=mac.rename(columns={"genotype": "cultivar"})
fl2=mac[["cultivar", "range", "column", "date_of_flowering", "days_to_flowering"]]

ksu=pd.read_csv("/content/drive/MyDrive/GPE/KSU/ksu_days_gdd_to_flowering.csv" , delimiter=",")
ksu=ksu.rename(columns={"pass": "column", "value": "days_to_flowering"})
fl1=ksu[["cultivar", "range", "column", "date_of_flowering", "days_to_flowering"]]

In [ ]:
dist=pd.read_csv("/content/drive/MyDrive/GPE/Cluster.txt", delimiter="\t")
a=dist.X
dist=dist.rename(columns={"X": "cultivar"})
dist=dist.dropna()

fl1=pd.merge(fl1, dist, on="cultivar", how='inner')

fl2=pd.merge(fl2, dist, on="cultivar", how='inner')

In [ ]:
ksw=pd.read_csv("/content/drive/MyDrive/GPE/KSU/ksu_weather.csv" , delimiter=",")
ksw['date']=pd.to_datetime(ksw.date)
day_0=pd.to_datetime('2016-06-17')
day_n=day_0 + pd.to_timedelta(20, 'days')

#for mean
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]
 
for j in range(fl1.shape[0]):
    li = (ksw['date']>= day_0) & (ksw['date'] <= day_n)
    df_1 = ksw[li]
    tmax.append(df_1['temp_max'].mean())
    tmin.append(df_1['temp_min'].mean())
    tmean.append(df_1['temp_mean'].mean())
    rhmax.append(df_1['rh_max'].mean())
    rhmin.append(df_1['rh_min'].mean())
    rhmean.append(df_1['rh_mean'].mean())
    vpd.append(df_1['vpd_mean'].mean())

fl1.insert(1,'temp_max_mean',tmax)
fl1.insert(2,'temp_min_mean',tmin)
fl1.insert(3,'temp_mean_mean',tmean)
fl1.insert(4,'rh_max_mean',rhmax)
fl1.insert(5,'rh_min_mean',rhmin)
fl1.insert(6,'rh_mean_mean',rhmean)
fl1.insert(7,'vpd_mean',vpd)

#for maximum
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]


for j in range(fl1.shape[0]):
    li = (ksw['date']>= day_0) & (ksw['date'] <= day_n)
    df_1 = ksw[li]
    tmax.append(df_1['temp_max'].max())
    tmin.append(df_1['temp_min'].max())
    tmean.append(df_1['temp_mean'].max())
    rhmax.append(df_1['rh_max'].max())
    rhmin.append(df_1['rh_min'].max())
    rhmean.append(df_1['rh_mean'].max())
    vpd.append(df_1['vpd_mean'].max())


fl1.insert(1,'temp_max_max',tmax)
fl1.insert(2,'temp_min_max',tmin)
fl1.insert(3,'temp_mean_max',tmean)
fl1.insert(4,'rh_max_max',rhmax)
fl1.insert(5,'rh_min_max',rhmin)
fl1.insert(6,'rh_mean_max',rhmean)
fl1.insert(7,'vpd_max',vpd)

#for minimum
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]

for j in range(fl1.shape[0]):
    li = (ksw['date']>= day_0) & (ksw['date'] <= day_n)
    df_1 = ksw[li]
    tmax.append(df_1['temp_max'].min())
    tmin.append(df_1['temp_min'].min())
    tmean.append(df_1['temp_mean'].min())
    rhmax.append(df_1['rh_max'].min())
    rhmin.append(df_1['rh_min'].min())
    rhmean.append(df_1['rh_mean'].min())
    vpd.append(df_1['vpd_mean'].min())

fl1.insert(1,'temp_max_min',tmax)
fl1.insert(2,'temp_min_min',tmin)
fl1.insert(3,'temp_mean_min',tmean)
fl1.insert(4,'rh_max_min',rhmax)
fl1.insert(5,'rh_min_min',rhmin)
fl1.insert(6,'rh_mean_min',rhmean)
fl1.insert(7,'vpd_min',vpd)

In [ ]:
mac=pd.read_csv("/content/drive/MyDrive/GPE/weather_data/mac_season_4_weather.csv" , delimiter=",")
mac['date']=pd.to_datetime(mac.date)
day_0=pd.to_datetime('2017-04-20')
day_n=day_0 + pd.to_timedelta(20, 'days')

#for mean
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]
 
for j in range(fl2.shape[0]):
    li = (mac['date']>= day_0) & (mac['date'] <= day_n)
    df_1 = mac[li]
    tmax.append(df_1['temp_max'].mean())
    tmin.append(df_1['temp_min'].mean())
    tmean.append(df_1['temp_mean'].mean())
    rhmax.append(df_1['rh_max'].mean())
    rhmin.append(df_1['rh_min'].mean())
    rhmean.append(df_1['rh_mean'].mean())
    vpd.append(df_1['vpd_mean'].mean())

fl2.insert(1,'temp_max_mean',tmax)
fl2.insert(2,'temp_min_mean',tmin)
fl2.insert(3,'temp_mean_mean',tmean)
fl2.insert(4,'rh_max_mean',rhmax)
fl2.insert(5,'rh_min_mean',rhmin)
fl2.insert(6,'rh_mean_mean',rhmean)
fl2.insert(7,'vpd_mean',vpd)

#for maximum
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]


for j in range(fl2.shape[0]):
    li = (mac['date']>= day_0) & (mac['date'] <= day_n)
    df_1 = mac[li]
    tmax.append(df_1['temp_max'].max())
    tmin.append(df_1['temp_min'].max())
    tmean.append(df_1['temp_mean'].max())
    rhmax.append(df_1['rh_max'].max())
    rhmin.append(df_1['rh_min'].max())
    rhmean.append(df_1['rh_mean'].max())
    vpd.append(df_1['vpd_mean'].max())


fl2.insert(1,'temp_max_max',tmax)
fl2.insert(2,'temp_min_max',tmin)
fl2.insert(3,'temp_mean_max',tmean)
fl2.insert(4,'rh_max_max',rhmax)
fl2.insert(5,'rh_min_max',rhmin)
fl2.insert(6,'rh_mean_max',rhmean)
fl2.insert(7,'vpd_max',vpd)

#for minimum
tmax = []
tmean=[]
tmin = []
rhmax=[]
rhmin=[]
rhmean=[]
vpd=[]

for j in range(fl2.shape[0]):
    li = (mac['date']>= day_0) & (mac['date'] <= day_n)
    df_1 = mac[li]
    tmax.append(df_1['temp_max'].min())
    tmin.append(df_1['temp_min'].min())
    tmean.append(df_1['temp_mean'].min())
    rhmax.append(df_1['rh_max'].min())
    rhmin.append(df_1['rh_min'].min())
    rhmean.append(df_1['rh_mean'].min())
    vpd.append(df_1['vpd_mean'].min())

fl2.insert(1,'temp_max_min',tmax)
fl2.insert(2,'temp_min_min',tmin)
fl2.insert(3,'temp_mean_min',tmean)
fl2.insert(4,'rh_max_min',rhmax)
fl2.insert(5,'rh_min_min',rhmin)
fl2.insert(6,'rh_mean_min',rhmean)
fl2.insert(7,'vpd_min',vpd)

In [ ]:
data=fl2.append(fl1)
new=data.drop(['cultivar', 'date_of_flowering', 'days_to_flowering'], axis=1)
X=new.to_numpy()
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

y=data[['days_to_flowering']]
y = np.asarray(y).flatten()

In [ ]:
#Hyperparameter tuning for the models
model = RandomForestRegressor()
max_depth = range(1, 20, 2)
n_estimators=range(20, 55, 5)
groups=data.cultivar
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), "for RF")

model = xgb.XGBRegressor()
n_estimators = range(10, 60, 5)
learning_rate = np.linspace(0,1,11)
max_depth = range(1, 20, 2)
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), "for XgB")

model = SGDRegressor()
alpha = np.arange(0.01, 1.0, 0.05)
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), "for SGD")

model = LassoLars()
alpha = np.arange(0.1, 1.0, 0.01)
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), "for LL")

model = Lasso()
alpha = np.arange(0.1, 1.0, 0.01)

param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), "for L")
model = Ridge()
alpha = np.arange(0.1, 1.0, 0.01)
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), "for R")

model = ElasticNet()
alpha = np.arange(0.01, 1.0, 0.05)
l1_ratio=np.arange(0, 1, 0.01)
param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_), "for EN")

Best: -11.358709 using {'max_depth': 9, 'n_estimators': 20} for RF
[17:13:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best: -11.370335 using {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 55} for XgB
Best: -13.803746 using {'alpha': 0.11} for SGD


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:582: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=1.748e-01, with an active set of 2 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:582: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=1.135e-01, with an active set of 3 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:582: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=1.135e-01, with an active set of 3 regressors, and the smallest cholesky pivot element 

Best: -13.724154 using {'alpha': 0.1} for LL
Best: -13.164801 using {'alpha': 0.1} for L
Best: -13.081240 using {'alpha': 0.9899999999999995} for R
Best: -13.038583 using {'alpha': 0.060000000000000005, 'l1_ratio': 0.99} for EN


In [ ]:
#Implementation of different algorithms

# 5 fold cross-validation for Random Forest
groups=data.cultivar
ns=5
cv = GroupKFold(n_splits=ns)
error=0
e=[]
for train_index, test_index in cv.split(X, y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('TRAIN :',X_train.shape, 'TEST :', X_test.shape)
    crf=RandomForestRegressor(max_depth=9, n_estimators=20).fit(X_train, y_train)

    pred=crf.predict(X_test)
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    error=error+math.sqrt(mean_squared_error(y_test, pred))

std= (statistics.stdev(e))
print('Root Mean square error for 5-fold CV RF',error/ns,'+-', std)

# 5 fold cross-validation for Stochastic Gradient Descent Regression

cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=SGDRegressor(alpha=0.11).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV SG:',error/ns, '+-', std)

# 5 fold cross-validation for Linear Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=LinearRegression().fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV LR:',error/ns, '+-', std)

# 5 fold cross-validation for XgBoost Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=xgb.XGBRegressor(booster='gbtree',importance_type='gain',learning_rate= 0.1, max_depth= 5, n_estimators= 55).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV XG:',error/ns, '+-', std)

# 5 fold cross-validation for LassoLars Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=LassoLars(alpha=0.1).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV LL:',error/ns, '+-', std)

# 5 fold cross-validation for Lasso Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=Lasso(alpha=0.1).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV L:',error/ns, '+-', std)

# 5 fold cross-validation for Ridge Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=Ridge(alpha=0.99).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV R:',error/ns, '+-', std)

# 5 fold cross-validation for ElasticNet Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=ElasticNet(alpha= 0.06, l1_ratio= 0.99, max_iter=5000).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    
std= (statistics.stdev(e))

print('Root Mean square error for 5-fold CV EN:',error/ns, '+-', std)




TRAIN : (189, 53) TEST : (48, 53)
TRAIN : (189, 53) TEST : (48, 53)
TRAIN : (190, 53) TEST : (47, 53)
TRAIN : (190, 53) TEST : (47, 53)
TRAIN : (190, 53) TEST : (47, 53)
Root Mean square error for 5-fold CV RF 12.064083670847563 +- 1.8444612525272905
Root Mean square error for 5-fold CV SG: 13.875197365359536 +- 0.9951118205406825
Root Mean square error for 5-fold CV LR: 15.57581548315363 +- 1.5015688849074071
[17:35:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:35:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:35:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:35:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:35:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:582: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 2 iterations, i.e. alpha=1.605e-01, with an active set of 2 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:582: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 4 iterations, i.e. alpha=1.004e-01, with an active set of 4 regressors, and the smallest cholesky pivot element being 1.490e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:582: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 3 iterations, i.e. alpha=1.099e-01, with an active set of 3 regressors, and the smallest cholesky pivot element 